In [ ]:
# !pip install chardet

In [81]:
import pymysql
import pandas as pd
import chardet
import os

In [82]:
fileName= 'csv_datas/CARD_SUBWAY_MONTH_202311.csv'
df = pd.read_csv(fileName, index_col=False)
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20231101,8호선,단대오거리,13482,13072,20231104
1,20231101,중앙선,원덕,372,362,20231104
2,20231101,중앙선,용문,2194,2097,20231104
3,20231101,중앙선,지평,72,71,20231104
4,20231101,우이신설선,4.19민주묘지,4515,4447,20231104


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18313 entries, 0 to 18312
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사용일자    18313 non-null  int64 
 1   노선명     18313 non-null  object
 2   역명      18313 non-null  object
 3   승차총승객수  18313 non-null  int64 
 4   하차총승객수  18313 non-null  int64 
 5   등록일자    18313 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 858.5+ KB


In [84]:
df.shape

(18313, 6)

In [85]:
def createDB(cur, conn):
    # 데이터베이스에 테이블을 생성
    scriptStr = "CREATE TABLE subwayTable(date int, route varchar(30), station varchar(50), boarding int, alighting int, regDate int)"
    cur.execute(scriptStr)
    conn.commit()

In [86]:
# insert시 df를 각 변수에서 가져 오느라 180만건 시행 했을 때 20분 이상 걸림

def insertDB(cur, conn, df):
    for i in range(df.shape[0]):
        insertSql = (
            "INSERT INTO subwayTable VALUES("
            + str(df.iloc[i][0])
            + ",'"
            + str(df.iloc[i][1])
            + "','"
            + str(df.iloc[i][2])
            + "',"
            + str(df.iloc[i][3])
            + ","
            + str(df.iloc[i][4])
            + ","
            + str(df.iloc[i][5])
            + ")"
        )
        cur.execute(insertSql)
    conn.commit()

In [87]:
# insert시 df를 최소한으로 핸들링 해서, 위와 동일한 환경에서 약 3분 미만으로 실행 완료 됨

def insertDB2(cur, conn, df):
    print(df.shape)
    for i in range(df.shape[0]):
        rowData = df.iloc[i]
        data0 = rowData['사용일자']
        data1 = rowData['노선명']
        data2 = rowData['역명']
        data3 = rowData['승차총승객수']
        data4 = rowData['하차총승객수']
        data5 = rowData['등록일자']
        insertSql = (
            f"INSERT INTO subwayTable VALUES( {data0}, '{data1}', '{data2}', {data3}, {data4}, {data5} )"
        )
        cur.execute(insertSql)
    conn.commit()

In [88]:
def readDB(cur, conn):
    cur.execute("SELECT * FROM subway_DB.subwayTable LIMIT 10")
    print("사용일자               노선명                   역명                승차총승객수            하차총승객수           등록일자  ")
    print("-----------------------------------------------------------------------------------------------------------------------")

    while True:
        row = cur.fetchone()
        if row == None:
            break
        # print(row)
        data1 = row[0]
        data2 = row[1]
        data3 = row[2]
        data4 = row[3]
        data5 = row[4]
        data6 = row[5]
        print(
            "%5s %15s %20s %20s %20s %20s" % (data1, data2, data3, data4, data5, data6)
        )
    
    conn.commit()

In [89]:
def checkFileEncoding(file_path):
    rawdata = open(file_path, "rb").readline()
    result = chardet.detect(rawdata)
    charenc = result["encoding"]
    charenc
    print("file_path : ", file_path)
    print("charenc : ", charenc)

def replace_in_file(file_path, old_str, new_str):
    rawdata = open(file_path, "rb").readline()
    result = chardet.detect(rawdata)
    charenc = result["encoding"]
    charenc

    fr = ""
    if charenc != "EUC-KR":
        # 파일 읽어들이기
        fr = open(file_path, "r", encoding="UTF-8")
    else:
        fr = open(file_path, "r", encoding="EUC-KR")

    lines = fr.readlines()
    fr.close()

    # old_str -> new_str 치환
    fw = open(file_path, "w", encoding="UTF-8")
    for line in lines:
        fw.write(line.replace(old_str, new_str))
    fw.close()


# 호출: file1.txt 파일에서 "" 없애기
# replace_in_file("data/CARD_SUBWAY_MONTH_2020.csv", '""', "")

In [90]:
# Step 1. DB 연결
conn = pymysql.connect(
    host="127.0.0.1", user="root", password="dongw2on", db="subway_DB", charset="utf8"
)

# Step 2. 커서 생성
cur = conn.cursor()

In [91]:
# Step 3. 데이터베이스에서 생성하기
createDB(cur, conn)

In [92]:
# Step 4. 데이터베이스에 입력하기
# insertDB(cur, conn, df)

In [93]:
# Step 5. 데이터베이스 읽기
# readDB(cur, conn)

In [94]:
datas = os.listdir('csv_datas')

In [95]:
datas

['CARD_SUBWAY_MONTH_202309.csv',
 'CARD_SUBWAY_MONTH_202308.csv',
 'CARD_SUBWAY_MONTH_2016.csv',
 'CARD_SUBWAY_MONTH_2017.csv',
 'CARD_SUBWAY_MONTH_2015.csv',
 'CARD_SUBWAY_MONTH_2022.csv',
 'CARD_SUBWAY_MONTH_2020.csv',
 'CARD_SUBWAY_MONTH_2021.csv',
 'CARD_SUBWAY_MONTH_2019.csv',
 'CARD_SUBWAY_MONTH_2018.csv',
 'CARD_SUBWAY_MONTH_202306.csv',
 'CARD_SUBWAY_MONTH_202307.csv',
 'CARD_SUBWAY_MONTH_202311.csv',
 'CARD_SUBWAY_MONTH_202305.csv',
 'CARD_SUBWAY_MONTH_202304.csv',
 'CARD_SUBWAY_MONTH_202310.csv',
 'CARD_SUBWAY_MONTH_202301.csv',
 'CARD_SUBWAY_MONTH_202303.csv',
 'CARD_SUBWAY_MONTH_202302.csv']

In [96]:
!pwd

/Users/dongweonshin/my_ws/GoogleCloudAI/edu/google_cloud/MySQL


In [24]:
# 반복해서 파일 읽어서 입력하기
for data in datas:
    print("fileName : ", data)
    replace_in_file('./csv_datas/'+data, ',""', "")
    # replace_in_file('./csv_datas/'+data, '",\n', '"\n')
    # checkFileEncoding('./csv_datas/'+data)

fileName :  CARD_SUBWAY_MONTH_202309.csv
fileName :  CARD_SUBWAY_MONTH_202308.csv
fileName :  CARD_SUBWAY_MONTH_2016.csv
fileName :  CARD_SUBWAY_MONTH_2017.csv
fileName :  CARD_SUBWAY_MONTH_2015.csv
fileName :  CARD_SUBWAY_MONTH_2022.csv
fileName :  CARD_SUBWAY_MONTH_2020.csv
fileName :  CARD_SUBWAY_MONTH_2021.csv
fileName :  CARD_SUBWAY_MONTH_2019.csv
fileName :  CARD_SUBWAY_MONTH_2018.csv
fileName :  CARD_SUBWAY_MONTH_202306.csv
fileName :  CARD_SUBWAY_MONTH_202307.csv
fileName :  CARD_SUBWAY_MONTH_202311.csv
fileName :  CARD_SUBWAY_MONTH_202305.csv
fileName :  CARD_SUBWAY_MONTH_202304.csv
fileName :  CARD_SUBWAY_MONTH_202310.csv
fileName :  CARD_SUBWAY_MONTH_202301.csv
fileName :  CARD_SUBWAY_MONTH_202303.csv
fileName :  CARD_SUBWAY_MONTH_202302.csv


In [97]:
# 반복해서 파일 읽어서 입력하기
for data in datas:
    # df2 = pd.read_csv('./csv_datas/'+data, index_col=False)
    # rawdata = open('./csv_datas/'+data, 'rb').readline()
    # result = chardet.detect(rawdata)
    # charenc = result['encoding']
    # charenc
    # print("encoding : ", charenc)
    print("fileName : ", data)
    
    # if charenc != "EUC-KR":
    #     df = pd.read_csv('./csv_datas/'+data, index_col=False)
    # else:
    #     df = pd.read_csv('./csv_datas/'+data, index_col=False, encoding='euc-kr')
        
    df = pd.read_csv('./csv_datas/'+data, index_col=False)
    insertDB2(cur, conn, df)

fileName :  CARD_SUBWAY_MONTH_202309.csv
(18302, 6)
fileName :  CARD_SUBWAY_MONTH_202308.csv
(18913, 6)
fileName :  CARD_SUBWAY_MONTH_2016.csv
(203435, 6)
fileName :  CARD_SUBWAY_MONTH_2017.csv
(208257, 6)
fileName :  CARD_SUBWAY_MONTH_2015.csv
(199380, 6)
fileName :  CARD_SUBWAY_MONTH_2022.csv
(219745, 6)
fileName :  CARD_SUBWAY_MONTH_2020.csv
(217055, 6)
fileName :  CARD_SUBWAY_MONTH_2021.csv
(219279, 6)
fileName :  CARD_SUBWAY_MONTH_2019.csv
(215769, 6)
fileName :  CARD_SUBWAY_MONTH_2018.csv
(212749, 6)
fileName :  CARD_SUBWAY_MONTH_202306.csv
(18214, 6)
fileName :  CARD_SUBWAY_MONTH_202307.csv
(18906, 6)
fileName :  CARD_SUBWAY_MONTH_202311.csv
(18313, 6)
fileName :  CARD_SUBWAY_MONTH_202305.csv
(18830, 6)
fileName :  CARD_SUBWAY_MONTH_202304.csv
(18211, 6)
fileName :  CARD_SUBWAY_MONTH_202310.csv
(18911, 6)
fileName :  CARD_SUBWAY_MONTH_202301.csv
(18797, 6)
fileName :  CARD_SUBWAY_MONTH_202303.csv
(18815, 6)
fileName :  CARD_SUBWAY_MONTH_202302.csv
(16994, 6)


In [74]:

cur.execute("SELECT count(*) AS CNT FROM subway_DB.subwayTable")
while(True):
    row = cur.fetchone()
    if row==None:
        break
    # print(row)
    data1 = row[0]
    print("%5s" %(data1))
conn.commit()

    0


In [56]:
# Step 6. 연결 끊기
conn.close()